In [ ]:
import pandas as pd
import vectorbt as vbt
from datetime import timedelta, datetime

df = pd.read_csv("data/EURUSD_1min.csv")
df = df.set_index('Time')
df.index = pd.to_datetime(df.index)


filtered_10_oclock = df.between_time("10:00", "10:00")
filtered_10_oclock


,Open,High,Low,Close,Volume
Time,,,,,
2023-01-02 10:00:00+00:00,1.06791,1.06791,1.06786,1.06786,23.0
2023-01-03 10:00:00+00:00,1.05476,1.05495,1.05470,1.05482,194.0
2023-01-04 10:00:00+00:00,1.06108,1.06133,1.06106,1.06126,125.0
2023-01-05 10:00:00+00:00,1.06234,1.06260,1.06224,1.06226,177.0
2023-01-06 10:00:00+00:00,1.05073,1.05160,1.05073,1.05142,208.0
...,...,...,...,...,...
2025-07-25 10:00:00+00:00,1.17352,1.17354,1.17334,1.17336,75.0
2025-07-28 10:00:00+00:00,1.16668,1.16722,1.16668,1.16719,244.0
2025-07-29 10:00:00+00:00,1.15766,1.15766,1.15755,1.15760,223.0


In [ ]:
df['Positions'] = 0.0
for current_idx, row in filtered_10_oclock.iterrows():
    idx_10_hour_ago = current_idx - timedelta(hours=10)
    idx_6hour_upfront = current_idx + timedelta(hours=6)
    max_high = df[idx_10_hour_ago:current_idx]['High'].max()
    
    if row['High'] >= max_high:
        df.loc[current_idx : idx_6hour_upfront, 'Positions'] = 1.0

df[df['Positions'] == 1]

In [ ]:
entries = df['Positions'] > 0
exits = df['Positions'] == 0
exits

In [ ]:
pf = vbt.Portfolio.from_signals(
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close'],
    entries=entries,
    exits=exits,
    size=1,
    size_type="percent",
    fees=0,
    slippage=0,
    init_cash=1000,
    freq="1min"
)

In [ ]:
pf.stats()